In [ ]:
# jupyter_vim
# docker-compose-rw.yaml automatically spins up containers (see generator service) that publishes messages to the kafka topic

In [ ]:
! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic=message --bootstrap-servers=kafka-broker:9092 --schema='{"message": "VARCHAR","created_at": "TIMESTAMP", "id": "VARCHAR"}' --source='kafka'

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True
topic = "message"  # same as source name

In [ ]:
with conn.cursor() as cur:
    cur.execute(f"SELECT kafka_timestamp, created_at, message, id FROM {topic};")
    values = cur.fetchall()

In [ ]:
df = pd.DataFrame(values, columns=["timestamp", "created_at", "message", "id"])
df.tail(5)

In [ ]:
df.timestamp.dt.floor("s").sort_values().unique()

In [ ]:
df.created_at.dt.floor("s").sort_values().unique()

In [ ]:
# create materialized view
with conn.cursor() as cur:
    cur.execute(f"DROP MATERIALIZED VIEW IF EXISTS {topic}_view")
    cur.execute(
        f"""
        CREATE MATERIALIZED VIEW {topic}_view AS
        SELECT
            date_trunc('second', created_at) AS window_start,
            SUM(LENGTH(message)) AS total_characters,
            count(*) AS total_messages
        FROM
            {topic} 
        GROUP BY
            window_start
        """
    )

In [ ]:
with conn.cursor() as cur:
    cur.execute(f"SELECT * FROM {topic}_view ORDER BY window_start;")
    values = cur.fetchall()
pd.DataFrame(
    values, columns=["timestamp", "total_characters", "total_lines"]
).sort_values("timestamp")

In [ ]:
import duckdb

con = duckdb.connect()
con.sql(
    f"""
    INSTALL postgres;
    LOAD postgres;  
    ATTACH 'host=risingwave port=4566 dbname=dev user=root' AS db (TYPE postgres);  
"""
)
con.sql("SHOW DATABASES;")

In [ ]:
# TODO
# con.sql(f"SELECT * FROM db.public.{topic}_view ORDER BY window_start;")